<a href="https://colab.research.google.com/github/tomaszes19/tomaszes19/blob/main/Analiza_sentymentu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pobieranie recenzji ze strony Metacritic dla trzech gier PC - Cyberpunk 2077, Wiedźmin 3 i Red Dead Redemption 2

In [ ]:
# Funkcja do pobierania recenzji
from requests_html import HTMLSession
import json
import pandas as pd

# otworzenie sesji HTML
sess = HTMLSession()

# funkcja pobierajaca dane ze strony Metacritic
def get_reviews(url):

    # lista do gromadzenia danych ze strony
    reviews_list = {"reviews":[], 'url':url}
    res = sess.get(url)
    html = res.html

    reviews = html.find('.product_reviews', first=True).find('[class="review_section"]')
    counter = 0

    # pętla do pobierania informacji dla określonych tagów HTML
    for review in reviews:
            try:
                review_body = review.find('.blurb_expanded', first=True).text
            except AttributeError:
                review_body = review.find('.review_body', first=True).text

            reviews_list['reviews'].append({
                "author" : review.find(".name", first=True).text,
                "review" : review_body.replace("\n", " "),
                "date" : review.find(".date", first=True).text,
                "score" : review.find(".review_grade", first=True).text
                })
            counter += 1
    print(counter)

    next_page = html.find(".flipper.next", first=True)

    if next_page:
        try:
            next_page_url = next_page.absolute_links.pop()
            reviews_list['reviews'].extend(get_reviews(next_page_url)['reviews'])
        except KeyError:
            pass

    return reviews_list

Utworzenie lokalizacji do zapisu plików z recenzjami

In [ ]:
# save a JSON file with selected EN words
from google.colab import drive
import json

drive.mount('/content/drive')

In [ ]:
!cd '/content/drive/My Drive/analiza_sentymentu'

Wykonanie funkcji get_reviews na poszczególnych adresach URL

In [ ]:
# Cyberpunk 2077 (CP2077)
url_cp2077 = """https://www.metacritic.com/game/pc/cyberpunk-2077/user-reviews?sort-by=date&num_items=100"""

with open('reviews_cp2077.json', 'w') as f:
    json.dump(get_reviews(url_cp2077), f)

In [ ]:
# Wiedźmin 3 (W3)
url_w3 = """https://www.metacritic.com/game/pc/the-witcher-3-wild-hunt/user-reviews?sort-by=date&num_items=100"""

with open('reviews_w3.json', 'w') as f:
    json.dump(get_reviews(url_w3), f)

In [ ]:
# Red Dead Redemption 2 (RDR2)
url_rdr2 = """https://www.metacritic.com/game/pc/red-dead-redemption-2/user-reviews?sort-by=most-helpful&num_items=100"""

with open('reviews_rdr2.json', 'w') as f:
    json.dump(get_reviews(url_rdr2), f)

Przygotowanie danych do trenowania i testowania modelu



In [ ]:
# Pobranie plików z recenzjami
from google.colab import files
uploaded_test_cp = files.upload()
uploaded_test_w3 = files.upload()
uploaded_test_rdr2 = files.upload()

In [ ]:
import json

# import pliku JSON dla CP2077
with open('reviews_cp.json', 'r') as f:
    reviews_cp = json.load(f)

# import pliku JSON dla W3
with open('reviews_w3.json', 'r') as f:
    reviews_w3 = json.load(f)

with open('reviews_rdr2.json', 'r') as f:
    reviews_rdr2 = json.load(f)

In [ ]:
# połączenie zawartości plików JSON w jeden słownik (dict)
reviews = reviews_cp['reviews'] + reviews_w3['reviews'] + reviews_rdr2['reviews']

Funkcje przygotowujące dane do modelu

In [ ]:
# Funkcja rozpoznająca język tekstu, żeby wybrać jedynie recenzje w języku angielskim
from langdetect import detect

def detect_language(text):
    try:
        language = detect(text)
        return language
    except:
        return None

In [ ]:
# utworzenie słownika dla recenzji po angielsku
reviews_en = {"reviews":[]}
counter = 0

# pętla wykonująca funkcję detect_language i dodająca recenzje w języku angielskim do nowego słownika
for rev in reviews:
  rev_en = detect_language(rev['review'])
  if rev_en == 'en':
    rev_en_dict = {'review_en' : rev_en}
    rev_with_en = {**rev, **rev_en_dict}
    reviews_en['reviews'].append(rev_with_en)